In [2]:
from podpac.core.coordinates import Coordinates, clinspace, UniformCoordinates1d, StackedCoordinates
from podpac.core.data import interpolate
from podpac.core.data.interpolate import (Interpolation, InterpolationException,
                                          Interpolator, INTERPOLATION_METHODS,
                                          INTERPOLATION_SHORTCUTS, NearestNeighbor)
from podpac.core.data.datasource import DataSource
from podpac.core.units import UnitsDataArray
from podpac.core.data.test.test_datasource import MockDataSource, MockArrayDataSource, MockNonuniformDataSource

import numpy as np

In [56]:
np.datetime64('2018-01-09')- np.datetime64('2018-01-07')

numpy.timedelta64(2,'D')

In [ ]:
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]], dims=['lat', 'lon'])

interp = Interpolation('nearest_preview')

srccoords, srccoords_index = srccoords.intersect(reqcoords, outer=True, return_indices=True)
coords, cidx = interp.select_coordinates(srccoords, srccoords_index, reqcoords)

In [ ]:
# test when selection is applied serially
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]], dims=['lat', 'lon'])

interp = Interpolation({
    'lat': 'nearest_preview',
    'lon': 'nearest_preview'
})

srccoords, srccoords_index = srccoords.intersect(reqcoords, outer=True, return_indices=True)
coords, cidx = interp.select_coordinates(srccoords, srccoords_index, reqcoords)

In [ ]:
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([([0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5])], dims=['lat_lon'])
srccoords, srccoords_index = srccoords.intersect(reqcoords, outer=True, return_indices=True)

for src_dim, idx in zip(srccoords, srccoords_index):
    print(src_dim)
    print(idx)

In [ ]:
len(reqcoords['lat'])
np.all(coords['lat'].coordinates == np.array([0, 2, 4]))

### Unstacked Coordinates

In [ ]:
class TestInterp(Interpolator):
    dims_supported = ['lat', 'lon']
    def interpolate(self, udims, source_coordinates, source_data, requested_coordinates, output_data):
        output_data = source_data
        return source_coordinates, source_data, output_data

# test if source and data are both length one
reqcoords = Coordinates([[-.5, 1.5, 3.5], [.5, 2.5, 4.5]], dims=['lat', 'lon'])
srccoords = Coordinates([[0, 2, 4], [0, 3, 4]], dims=['lat', 'lon'])
srcdata = UnitsDataArray(np.random.rand(3, 3),
                         coords=[srccoords[c].coordinates for c in srccoords],
                         dims=srccoords.dims)
outdata = UnitsDataArray(np.zeros(srcdata.shape),
                         coords=[reqcoords[c].coordinates for c in reqcoords],
                         dims=reqcoords.dims)

interp = Interpolation({('lat', 'lon'): {'method': 'test', 'interpolators': [TestInterp]}})

srccoords, srcdata, outdata = interp.interpolate(srccoords, srcdata, reqcoords, outdata)

### Stacked Coordinates

In [ ]:
class TestInterp(Interpolator):
    dims_supported = ['lat', 'lon']
    def interpolate(self, udims, source_coordinates, source_data, requested_coordinates, output_data):
        output_data = source_data
        return source_coordinates, source_data, output_data

# test if source and data are both length one
reqcoords = Coordinates([(np.linspace(-25, 25, 51), np.linspace(-25, 25, 51))], dims=['lat_lon'])
srccoords = Coordinates([(np.linspace(-20, 30, 51), np.linspace(-20, 30, 51))], dims=['lat_lon'])

data = np.random.rand(21)
lat = np.linspace(-25, 25, 21)
lon = np.linspace(-25, 25, 21)

# create podpac coordinates for data
stacked = Coordinates([(lat, lon)], ['lat_lon'])

# create node for data source
interp = Interpolation({('lat', 'lon'): {'method': 'test', 'interpolators': [TestInterp]}})
node = DataSource(source=data, native_coordinates=stacked, interpolation=interp)
node.native_coordinates


# srccoords, srcdata, outdata = interp.interpolate(srccoords, srcdata, reqcoords, outdata)

In [ ]:
arr = node.create_output_array(node.native_coordinates)
arr.reindex(method='nearest', lat=np.linspace(-10, 10, 10))

In [ ]:
node.native_coordinates['lat']

In [ ]:
# source_coordinates = Coordinates([clinspace(-25, 25, 51), clinspace(-25, 25, 51)], dims=['lat', 'lon'])
source_coordinates = Coordinates([(np.linspace(-25, 25, 51), np.linspace(-25, 25, 51))], dims=['lat_lon'])
requested_coordinates = Coordinates([clinspace(-15, 15, 51), clinspace(-15, 15, 51)], dims=['lat', 'lon'])

source_coordinates, source_coordinates_index = source_coordinates.intersect(requested_coordinates, outer=True, return_indices=True)

new_coords =[]
new_coords_idx = []
for dim, idx in zip(source_coordinates, source_coordinates_index):
    if isinstance(source_coordinates[dim], StackedCoordinates):
        
    
    src_coords = source_coordinates[dim]
    dst_coords = requested_coordinates[dim]

    if isinstance(dst_coords, UniformCoordinates1d):
        dst_start = dst_coords.start
        dst_stop = dst_coords.stop
        dst_delta = dst_coords.step
    else:
        dst_start = dst_coords.coordinates[0]
        dst_stop = dst_coords.coordinates[-1]
        dst_delta = (dst_stop-dst_start) / (dst_coords.size - 1)

    if isinstance(src_coords, UniformCoordinates1d):
        src_start = src_coords.start
        src_stop = src_coords.stop
        src_delta = src_coords.step
    else:
        src_start = src_coords.coordinates[0]
        src_stop = src_coords.coordinates[-1]
        src_delta = (src_stop-src_start) / (src_coords.size - 1)

    ndelta = max(1, np.round(dst_delta / src_delta))

    c = UniformCoordinates1d(src_start, src_stop, ndelta*src_delta, **src_coords.properties)

    if isinstance(idx, slice):
        idx = slice(idx.start, idx.stop, int(ndelta))
    else:
        idx = slice(idx[0], idx[-1], int(ndelta))

    new_coords.append(c)
    new_coords_idx.append(idx)

In [ ]:
isinstance(source_coordinates['lat_lon'], StackedCoordinates)

In [ ]:
[c.intersect(requested_coordinates, outer=True, return_indices=True)[1] for c in stacked._coords]

In [ ]:
interp = Interpolation({('lat',): 'nearest', 'lon': 'bilinear'}, COORDINATES)

In [ ]:
# create a few dummy interpolators that handle certain dimensions
# (can_select is defined by default to look at dims_supported)
class TimeLat(Interpolator):
    dims_supported=['time', 'lat']

class LatLon(Interpolator):
    dims_supported=['lat', 'lon']
    
class Lon(Interpolator):
    dims_supported=['lon']


# set up a strange interpolation definition
# we want to interpolate (lat, lon) first, then after (time, alt)
interp = Interpolation({
    ('lat', 'lon'): {
        'method': 'myinterp',
        'interpolators': [LatLon, TimeLat]
    },
    ('time', 'alt'): {
        'method': 'myinterp',
        'interpolators': [TimeLat, Lon]
    }
})

reqcoords = Coordinates([[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]], dims=['lat', 'lon', 'time', 'alt'])
srccoords = Coordinates([[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]], dims=['lat', 'lon', 'time', 'alt'])
interpolator_queue = interp._select_interpolator_queue(reqcoords, srccoords, 'can_select', strict=True)

In [3]:
source = np.random.rand(5,6)
coords_src = Coordinates([np.linspace(0, 10, 5), clinspace('2018-01-01', '2018-01-09', 6)], dims=['lat', 'time'])
node = MockArrayDataSource(source=source, native_coordinates=coords_src, interpolation={
    'method': 'nearest',
    'params': {
        'space_tolerance': 1.1,
        'time_tolerance': np.timedelta64(1, 'D')
    }
})

coords_dst = Coordinates([[1, 1.2, 1.5, 5, 9], clinspace('2018-01-01', '2018-01-09', 3)], dims=['lat', 'time'])
# output = node.eval(coords_dst)

In [3]:
interpolator = Interpolator()

coords = Coordinates([clinspace(0, 10, 5), clinspace(0, 10, 5)], dims=['lat', 'lon'])
coords_two = Coordinates([clinspace(0, 10, 5)], dims=['lat'])

In [6]:
a = [1,2,3]
tuple(a)

(1, 2, 3)

In [5]:
node.native_coordinates

Coordinates
	lat: ArrayCoordinates1d(lat): Bounds[0.0, 10.0], N[5], ctype['midpoint']
	time: UniformCoordinates1d(time): Bounds[2018-01-01, 2018-01-09], N[9], ctype['midpoint']

In [15]:
clinspace('2018-01-01', '2018-01-09', 4)

UniformCoordinates1d(?): Bounds[2018-01-01, 2018-01-09], N[5], ctype['midpoint']